In [26]:
from sklearn.manifold import TSNE
import torch
from time import time
import pickle
import torch
import numpy as np
import json

import os
import torchvision.transforms as T
from collections import Counter

import torchvision.datasets as datasets
import random
import torch.utils.data as Data
import gc
from sklearn.cluster import KMeans
import numpy as np


def sim_layer(unitsA, unitsB):
    u = list(set(unitsA) | set(unitsB))
    i = list(set(unitsA) & set(unitsB))
    return len(i)/len(u)

def similarity(pathsA, pathsB):
    sim = 0
    sims = []
    for layer in range(len(pathsA)):
        sim += sim_layer(pathsA[layer], pathsB[layer])
        sims.append(sim_layer(pathsA[layer], pathsB[layer]))
    sim = sim/len(pathsA)
    return sim, sims

def get_random(samples, num_picked, badboys=[]):
    picked = []
    indexs = []
#     print(len(samples))
    while len(picked) < num_picked:
        index = random.randint(0, len(samples)-1)
        if index not in indexs and index not in badboys:
            indexs.append(index)
            picked.append(samples[index])
    return picked

def get_similarity(samples1, samples2):
    s_all = 0
    sims_avg = np.array([0.0 for _ in range(len(paths[0]))])
    for s1 in samples1:
        p1 = paths[s1]
        for s2 in samples2:
            p2 = paths[s2]
            s, sims = similarity(p1, p2)
#             print(sims)
            s_all += s
            sims_avg = sims_avg + np.array(sims)       
    s_avg = s_all / (len(samples1) * len(samples2))
    return s_avg, sims_avg/(len(samples1) * len(samples2))

def get_similarity_samples_cluster(samples1, path2):
    s_all = 0
    num = 0
    sims_avg = np.array([0.0 for _ in range(len(paths[0]))])
    for s1 in samples1:
        p1 = paths[s1]
        p2 = path2
        s, sims = similarity(p1, p2)
#             print(sims)
        s_all += s
        num += 1
        sims_avg = sims_avg + np.array(sims)       
    s_avg = s_all / num
    return s_avg, sims_avg/num

data_path = "LRP_path/imagenet_vgg16_bn_lrp_path_threshold0.7_train.pkl"
with open(data_path, 'rb') as fr:
    paths = pickle.load(fr)
    
picked_samples_classes = [[] for _ in range(10)]
num_cluster = 4
for cla in range(10):
    for clu in range(num_cluster):
        l = "{}_{}".format(cla, clu)
        picked_samples_fname = "cluster_paths/vgg16_bn_binary_cluster/num_cluster{}_class{}_cluster{}_picked_samples.pkl".format(num_cluster, cla, clu)
        with open(picked_samples_fname, "rb") as f:
            unpickler = pickle.Unpickler(f)
            picked_samples = unpickler.load()
        picked_samples_classes[cla].append(picked_samples)
        


# samples_path with cluster_path

In [10]:
picked_num = 5
sims_avg = np.array([0.0 for _ in range(len(paths[0]))])
s_all = 0
num = 0
s_dict = {}
for cla in range(10):
    for clu in range(num_cluster):
        samples1 = get_random(picked_samples_classes[cla][clu], picked_num, [])
        samples2 = get_random(picked_samples_classes[cla][clu], picked_num, samples1)
        path_fname = "cluster_paths/convmnist_binary_cluster/num_cluster{}_threshold{}_class{}_cluster{}_paths.pkl".format(num_cluster, 0.4, cla, clu)
        with open(path_fname, "rb") as f:
            unpickler = pickle.Unpickler(f)
            path2 = unpickler.load()[0]
        s, sims = get_similarity_samples_cluster(samples1, path2)
        s_dict["{}_{}".format(cla, clu)] = s
        s_all += s
        sims_avg = sims_avg + sims 
        num += 1
print(s_all/num)
print(sims_avg/num)

7
7


IndexError: list index out of range

In [25]:
# picked_num = 100
# sims_avg = np.array([0.0 for _ in range(len(paths[0]))])
# s_all = 0
# num = 0
# for cla in range(10):
#     for clu in range(num_cluster):
#         for clu2 in range(num_cluster):
#             if clu != clu2:
#                 samples1 = get_random(picked_samples_classes[cla][clu2], picked_num, [])
#                 path_fname = "cluster_paths/convmnist_binary_cluster/num_cluster{}_threshold{}_class{}_cluster{}_paths.pkl".format(num_cluster, 0.4, cla, clu)
#                 with open(path_fname, "rb") as f:
#                     unpickler = pickle.Unpickler(f)
#                     path2 = unpickler.load()[0]
#                 s, sims = get_similarity_samples_cluster(samples1, path2)
#         #         print(s)
#                 s_all += s
#                 sims_avg = sims_avg + sims 
#                 num += 1
# print(s_all/num)
# print(sims_avg/num)

0.6334631036457855
[0.6839908  0.76361491 0.4527836 ]


In [26]:
# picked_num = 100
# sims_avg = np.array([0.0 for _ in range(len(paths[0]))])
# s_all = 0
# num = 0
# for cla in range(10):
#     for clu in range(num_cluster):
#         for clu2 in range(num_cluster):
#             if clu != clu2:
#                 path_fname = "cluster_paths/convmnist_binary_cluster/num_cluster{}_threshold{}_class{}_cluster{}_paths.pkl".format(num_cluster, 0.4, cla, clu2)
#                 path_fname = "cluster_paths/convmnist_binary_cluster/num_cluster{}_threshold{}_class{}_cluster{}_paths.pkl".format(num_cluster, 0.4, cla, clu)
#                 with open(path_fname, "rb") as f:
#                     unpickler = pickle.Unpickler(f)
#                     path2 = unpickler.load()[0]
#                 s, sims = get_similarity_samples_cluster(samples1, path2)
#         #         print(s)
#                 s_all += s
#                 sims_avg = sims_avg + sims 
#                 num += 1
# print(s_all/num)
# print(sims_avg/num)

In [27]:
# picked_num = 10
# sims_avg = np.array([0.0 for _ in range(len(paths[0]))])
# s_all = 0
# num = 0
# for cla in range(10):
#     for cla2 in range(10):
#         if cla != cla2:
#             for clu in range(num_cluster):
#                 for clu2 in range(num_cluster):
                
#                     samples1 = get_random(picked_samples_classes[cla][clu2], picked_num, [])
#                     path_fname = "cluster_paths/convmnist_binary_cluster/num_cluster{}_threshold{}_class{}_cluster{}_paths.pkl".format(num_cluster, 0.4, cla, clu)
#                     with open(path_fname, "rb") as f:
#                         unpickler = pickle.Unpickler(f)
#                         path2 = unpickler.load()[0]
#                     s, sims = get_similarity_samples_cluster(samples1, path2)
#             #         print(s)
#                     s_all += s
#                     sims_avg = sims_avg + sims 
#                     num += 1
# print(s_all/num)
# print(sims_avg/num)

0.6584000462138969
[0.70624392 0.77821609 0.49074013]


# similarity of inter-cluster of different classes

In [32]:
picked_num = 10
sims_avg = np.array([0.0 for _ in range(len(paths[0]))])
s_all = 0
s_array = []
num = 0
for cla in range(10):
    for cla2 in range(10):
#         print("cla1, cla2", cla, cla2)
        if cla != cla2:
            for clu in range(num_cluster):
                for clu2 in range(num_cluster): 
                    if len(picked_samples_classes[cla][clu]) < picked_num or len(picked_samples_classes[cla2][clu2]) < picked_num:
                        break
                    samples1 = get_random(picked_samples_classes[cla][clu], picked_num, [])
                    samples2 = get_random(picked_samples_classes[cla2][clu2], picked_num, [])
                    s, sims = get_similarity(samples1, samples2)
                    s_all += s
                    s_array.append(s)
                    sims_avg = sims_avg + sims 
                    num += 1
print(s_all/num)
print(sims_avg/num)
print("max", max(s_array))
print("min", min(s_array))
print("mean", np.mean(s_array))

cla1, cla2 0 0
cla1, cla2 0 1
cla1, cla2 0 2
cla1, cla2 0 3
cla1, cla2 0 4
cla1, cla2 0 5
cla1, cla2 0 6
cla1, cla2 0 7
cla1, cla2 0 8
cla1, cla2 0 9
cla1, cla2 1 0
cla1, cla2 1 1
cla1, cla2 1 2
cla1, cla2 1 3
cla1, cla2 1 4
cla1, cla2 1 5
cla1, cla2 1 6
cla1, cla2 1 7
cla1, cla2 1 8
cla1, cla2 1 9
cla1, cla2 2 0
cla1, cla2 2 1
cla1, cla2 2 2
cla1, cla2 2 3
cla1, cla2 2 4
cla1, cla2 2 5
cla1, cla2 2 6
cla1, cla2 2 7
cla1, cla2 2 8
cla1, cla2 2 9
cla1, cla2 3 0
cla1, cla2 3 1
cla1, cla2 3 2
cla1, cla2 3 3
cla1, cla2 3 4
cla1, cla2 3 5
cla1, cla2 3 6
cla1, cla2 3 7
cla1, cla2 3 8
cla1, cla2 3 9
cla1, cla2 4 0
cla1, cla2 4 1
cla1, cla2 4 2
cla1, cla2 4 3
cla1, cla2 4 4
cla1, cla2 4 5
cla1, cla2 4 6
cla1, cla2 4 7
cla1, cla2 4 8
cla1, cla2 4 9
cla1, cla2 5 0
cla1, cla2 5 1
cla1, cla2 5 2
cla1, cla2 5 3
cla1, cla2 5 4
cla1, cla2 5 5
cla1, cla2 5 6
cla1, cla2 5 7
cla1, cla2 5 8
cla1, cla2 5 9
cla1, cla2 6 0
cla1, cla2 6 1
cla1, cla2 6 2
cla1, cla2 6 3
cla1, cla2 6 4
cla1, cla2 6 5
cla1, cla2

# similarity of intra-cluster

In [33]:
picked_num = 20
sims_avg = np.array([0.0 for _ in range(len(paths[0]))])
s_all = 0
num = 0
s_dict = {}
for cla in range(10):
    for clu in range(num_cluster):
        if len(picked_samples_classes[cla][clu]) < picked_num or len(picked_samples_classes[cla2][clu2]) < picked_num:
            break
        samples1 = get_random(picked_samples_classes[cla][clu], picked_num, [])
        samples2 = get_random(picked_samples_classes[cla][clu], picked_num, samples1)
        s, sims = get_similarity(samples1, samples2)
        s_dict["{}_{}".format(cla, clu)] = s
        s_all += s
        sims_avg = sims_avg + sims 
        num += 1
print(s_all/num)
print(sims_avg/num)
print(s_dict)

0.5459249085999283
[0.53437095 0.53865626 0.52708599 0.55141106 0.52784171 0.53334347
 0.55591333 0.53295616 0.54729817 0.56769121 0.54404282 0.56432654
 0.57208617]
{'0_0': 0.6056873324929635, '0_1': 0.6130801558054002, '0_2': 0.6617865410564999, '0_3': 0.5250708970970669, '1_0': 0.6027246068810768, '1_1': 0.6970793196373224, '1_2': 0.5061038664171295, '1_3': 0.6592352907838332, '2_0': 0.5225304399537738, '2_1': 0.5212114502646282, '2_2': 0.510060592551235, '2_3': 0.5783574020821443, '3_0': 0.5207383091731629, '3_1': 0.517130036993969, '3_2': 0.48005807874107737, '3_3': 0.5516010591514484, '4_0': 0.465737059396347, '4_1': 0.5156631090511905, '4_2': 0.5207439338775574, '4_3': 0.44094456109922364, '5_0': 0.43121185720156424, '5_1': 0.4641086450297034, '5_2': 0.4702674816162215, '5_3': 0.4386412630097257, '6_0': 0.4365751233749464, '6_1': 0.490163980638621, '6_2': 0.5326766184735288, '6_3': 0.5039377159792987, '7_0': 0.6158631708960008, '7_1': 0.6160206584121555, '7_2': 0.562682319290935

In [30]:
picked_num = 20
sims_avg = np.array([0.0 for _ in range(len(paths[0]))])
s_all = 0
num = 0
s_dict = {}
for cla in range(10):
    for clu in range(num_cluster):
        samples1 = get_random(picked_samples_classes[cla][clu], picked_num, [])
        samples2 = get_random(picked_samples_classes[cla][clu], picked_num, samples1)
        s, sims = get_similarity(samples1, samples2)
        s_dict["{}_{}".format(cla, clu)] = s
        s_all += s
        sims_avg = sims_avg + sims 
        num += 1
print(s_all/num)
print(sims_avg/num)
print(s_dict)

0.5465098825843722
[0.53436335 0.53916049 0.52914786 0.55445695 0.53068046 0.53463495
 0.55710649 0.53164919 0.54519196 0.5653617  0.5434667  0.56396414
 0.57544423]
{'0_0': 0.6121273392968356, '0_1': 0.6251347342603599, '0_2': 0.670103312166017, '0_3': 0.5285846212284557, '1_0': 0.5877626484248325, '1_1': 0.7193423950565897, '1_2': 0.5148953848845695, '1_3': 0.6288467402527598, '2_0': 0.5392567741035884, '2_1': 0.5419363057287281, '2_2': 0.5006832718622948, '2_3': 0.5557763484176094, '3_0': 0.5109393277515337, '3_1': 0.5048638913685174, '3_2': 0.4745915145251171, '3_3': 0.553485217408151, '4_0': 0.4708147523919613, '4_1': 0.5133039307707645, '4_2': 0.5230329371401319, '4_3': 0.456799940814739, '5_0': 0.45670670618408893, '5_1': 0.4664609401831165, '5_2': 0.47007574149212045, '5_3': 0.43203382970671356, '6_0': 0.4378045046075452, '6_1': 0.46751973710538336, '6_2': 0.5392944544762047, '6_3': 0.517916547420161, '7_0': 0.6064531045316682, '7_1': 0.6259543578514797, '7_2': 0.56523652909988

# similarity of inter-cluster

In [31]:
pick_num = 20
sims_avg = np.array([0.0 for _ in range(len(paths[0]))])
s_all = 0
num = 0
s_array = []
for cla in range(10):
    for clu in range(num_cluster):
        for clu2 in range(num_cluster):
            if clu != clu2:
                samples1 = get_random(picked_samples_classes[cla][clu], picked_num, [])
                samples2 = get_random(picked_samples_classes[clu][clu2], picked_num, [])
                s, sims = get_similarity(samples1, samples2)
                s_all += s
                s_array.append(s)
                sims_avg = sims_avg + sims 
                num += 1
print(s_all/num)
print(sims_avg/num)          
print("max", max(s_array))
print("min", min(s_array))
print("mean", np.mean(s_array))

0.33003026487146414
[0.34083939 0.35922467 0.35969687 0.38728214 0.35535685 0.36771186
 0.3974765  0.35725385 0.35175481 0.34657715 0.26439252 0.22968486
 0.17314197]
max 0.6196778714042624
min 0.20573534745935823
mean 0.3300302648714642


# draw more in paper

In [58]:
data_path = "LRP_path/mnist_convmnist_lrp_path_threshold0.8_train.pkl"
with open(data_path, 'rb') as fr:
    paths = pickle.load(fr)

picked_num = 50

picked_samples_classes = [[] for _ in range(10)]
num_cluster = 1
clu = 0
for cla in range(10):
    for clu in range(num_cluster):
        l = "{}_{}".format(cla, clu)
        print(l)
        picked_samples_fname = "cluster_paths/convmnist_binary_cluster/num_cluster{}_class{}_cluster{}_picked_samples.pkl".format(num_cluster, cla, clu)
        with open(picked_samples_fname, "rb") as f:
            unpickler = pickle.Unpickler(f)
            picked_samples = unpickler.load()
        picked_samples_classes[cla].append(picked_samples)

0_0
1_0
2_0
3_0
4_0
5_0
6_0
7_0
8_0
9_0


In [59]:
sims_avg = np.array([0.0 for _ in range(len(paths[0]))])
s_all = 0
s_array = []
num = 0
results_simCla = []
results_difCla = []
for cla in range(10):
    results = []
    for cla2 in range(cla, 10):
        print("{}_{}".format(cla, cla2))
        if cla == cla2:
            samples1 = get_random(picked_samples_classes[cla][0], picked_num, [])
            samples2 = get_random(picked_samples_classes[cla2][0], picked_num, samples1)
        else:
            samples1 = get_random(picked_samples_classes[cla][0], picked_num, [])
            samples2 = get_random(picked_samples_classes[cla2][0], picked_num, [])
        s, sims = get_similarity(samples1, samples2)
        results.append(round(s, 2))
        if cla2 == cla:
            results_simCla.append(round(s, 2))
        else:
            results_difCla.append(round(s, 2))
            
    print(results)
    
print(np.mean(results_difCla))
print(np.mean(results_simCla))

0_0
0_1
0_2
0_3
0_4
0_5
0_6
0_7
0_8
0_9
[0.7, 0.4, 0.35, 0.28, 0.39, 0.29, 0.42, 0.39, 0.42, 0.34]
1_1
1_2
1_3
1_4
1_5
1_6
1_7
1_8
1_9
[0.59, 0.25, 0.19, 0.33, 0.2, 0.28, 0.31, 0.28, 0.27]
2_2
2_3
2_4
2_5
2_6
2_7
2_8
2_9
[0.63, 0.49, 0.4, 0.46, 0.45, 0.41, 0.51, 0.5]
3_3
3_4
3_5
3_6
3_7
3_8
3_9
[0.68, 0.38, 0.56, 0.39, 0.38, 0.46, 0.49]
4_4
4_5
4_6
4_7
4_8
4_9
[0.57, 0.36, 0.43, 0.43, 0.41, 0.44]
5_5
5_6
5_7
5_8
5_9
[0.63, 0.4, 0.37, 0.44, 0.47]
6_6
6_7
6_8
6_9
[0.61, 0.39, 0.45, 0.42]
7_7
7_8
7_9
[0.56, 0.39, 0.46]
8_8
8_9
[0.64, 0.47]
9_9
[0.64]
0.39111111111111113
0.625


In [62]:
data_path = "LRP_path/mnist_convmnist_lrp_path_threshold0.9_train.pkl"
with open(data_path, 'rb') as fr:
    paths = pickle.load(fr)

picked_num = 50

picked_samples_classes = [[] for _ in range(10)]
num_cluster = 1
clu = 0
for cla in range(10):
    for clu in range(num_cluster):
#         l = "{}_{}".format(cla, clu)
#         print(l)
        picked_samples_fname = "cluster_paths/convmnist_binary_cluster/num_cluster{}_class{}_cluster{}_picked_samples.pkl".format(num_cluster, cla, clu)
        with open(picked_samples_fname, "rb") as f:
            unpickler = pickle.Unpickler(f)
            picked_samples = unpickler.load()
        picked_samples_classes[cla].append(picked_samples)

9_0
9_0
9_0
9_0
9_0
9_0
9_0
9_0
9_0
9_0


In [16]:
picked_num = 5
sims_avg = np.array([0.0 for _ in range(len(paths[0]))])
s_all = 0
s_array = []
num = 0
results_simCla = []
results_difCla = []
for cla in range(10):
    results = []
    for cla2 in range(cla, 10):
#         print("{}_{}".format(cla, cla2))
        p1 = []
        for l in picked_samples_classes[cla]:
            p1 = p1 + l
        p2 = []
        for l in picked_samples_classes[cla2]:
            p2 = p2 + l
#         p1 = [[].extend(l) for l in picked_samples_classes[cla]]
#         p2 = [[].extend(l) for l in picked_samples_classes[cla2]]
#         print(p1)
        if cla == cla2:
            if len(p1) < picked_num or len(p2) < picked_num:
                break
            samples1 = get_random(p1, picked_num, [])
            samples2 = get_random(p2, picked_num, samples1)
        else:
            if len(p1) < picked_num or len(p2) < picked_num:
                        break
            samples1 = get_random(p1, picked_num, [])
            samples2 = get_random(p2, picked_num, [])
        s, sims = get_similarity(samples1, samples2)
        results.append(round(s, 2))
        if cla2 == cla:
            results_simCla.append(round(s, 2))
        else:
            results_difCla.append(round(s, 2))
            
    print(results)
    
print(np.mean(results_difCla))
print(np.mean(results_simCla))

[0.63]
[]
[]
[]
[]
[]
[]
[]
[]
[]
nan
0.63


/home/xiaofei/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/xiaofei/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [68]:
data_path = "LRP_path/mnist_convmnist_lrp_path_threshold1.0_train.pkl"
with open(data_path, 'rb') as fr:
    paths = pickle.load(fr)

picked_num = 50

picked_samples_classes = [[] for _ in range(10)]
num_cluster = 1
clu = 0
for cla in range(10):
    for clu in range(num_cluster):
        l = "{}_{}".format(cla, clu)
#         print(l)
        picked_samples_fname = "cluster_paths/convmnist_binary_cluster/num_cluster{}_class{}_cluster{}_picked_samples.pkl".format(num_cluster, cla, clu)
        with open(picked_samples_fname, "rb") as f:
            unpickler = pickle.Unpickler(f)
            picked_samples = unpickler.load()
        picked_samples_classes[cla].append(picked_samples)

In [24]:
picked_num = 20
sims_avg = np.array([0.0 for _ in range(len(paths[0]))])
s_all = 0
s_array = []
num = 0
results_simCla = []
results_difCla = []
for cla in range(10):
    results = []
    for cla2 in range(cla, 10):
#         print("{}_{}".format(cla, cla2))
        if cla == cla2:
            samples1 = get_random(picked_samples_classes[cla][0], picked_num, [])
            samples2 = get_random(picked_samples_classes[cla2][0], picked_num, samples1)
        else:
            samples1 = get_random(picked_samples_classes[cla][0], picked_num, [])
            samples2 = get_random(picked_samples_classes[cla2][0], picked_num, [])
        s, sims = get_similarity(samples1, samples2)
        results.append(round(s, 2))
        if cla2 == cla:
            results_simCla.append(round(s, 2))
        else:
            results_difCla.append(round(s, 2))
            
    print(results)
    
print(np.mean(results_difCla))
print(np.mean(results_simCla))

[0.99, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98]
[0.99, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.97]
[0.99, 0.98, 0.98, 0.98, 0.98, 0.98, 0.98, 0.97]
[0.99, 0.98, 0.98, 0.98, 0.98, 0.97, 0.97]
[0.99, 0.98, 0.98, 0.98, 0.98, 0.97]
[0.98, 0.98, 0.98, 0.98, 0.97]
[0.99, 0.98, 0.98, 0.97]
[0.99, 0.99, 0.98]
[0.99, 0.98]
[0.99]
0.9786666666666665
0.9890000000000001
